In [1]:
from datascience import *
from datascience.predicates import are
path_data = '../../../../data/'
import numpy as np
import matplotlib
matplotlib.use('Agg')
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

from urllib.request import urlopen 
import re
from bancor3_simulator.protocol import BancorV3
from bancor3_simulator.core.settings import GlobalSettings
from decimal import Decimal
from fractions import Fraction
import pandas as pd

def read_url(url): 
    return re.sub('\\s+', ' ', urlopen(url).read().decode())

# Single-Sided Pool Tokens

The proposed upgrade fundamentally changes the meaning of pool tokens in AMMs. The AMM pool token concept, or “smart token”, was introduced by Bancor with its initial whitepaper and persists essentially unaltered to the present day. The concept was built around the assumption that liquidity providers would participate in both sides of the pool; however, with the introduction of single-sided staking by Bancor, a revision of the pool token idea is overdue. Bancor v2.1 still makes use of traditional pool tokens, although their role in the system is permanently obscured from view. A plethora of auxiliary mechanisms are required to support single sided staking, which can largely be replaced by a purpose-built system that assumes the contribution of only one token, rather than two or more.

**Single-Sided Pool Tokens** express themselves in the simulator code via the `Transaction` class definition.

Note that the `Transaction` itself can only track one `tkn_name` balance at a time.

Even so, notice that the ledger automatically pairs the single-sided token with BNT, providing a true ***utility*** behavior as a global medium of exchange between all other pool tokens.

In [2]:
from dataclasses import dataclass


@dataclass
class Transaction(GlobalSettings):
    timestep: int = 0
    vault_tkn: Decimal = Decimal("0")
    erc20contracts_bntkn: Decimal = Decimal("0")
    staked_tkn: Decimal = Decimal("0")
    trading_enabled: bool = False
    bnt_trading_liquidity: Decimal = Decimal("0")
    tkn_trading_liquidity: Decimal = Decimal("0")
    trading_fee: Decimal = Decimal("0.005")
    bnt_funding_limit: Decimal = Decimal("0")
    bnt_remaining_funding: Decimal = Decimal("0")
    bnt_funding_amount: Decimal = Decimal("0")
    external_protection_vault: Decimal = Decimal("0")
    spot_rate: Decimal = Decimal("0")
    ema_rate: Decimal = Decimal("0")
    ema_last_updated: Decimal = Decimal("0")

    @property
    def is_price_stable(self):
        """Computes the deviation between the spot and ema bnt/tkn rates. True if the deviation is less than 1%."""
        return Decimal("0.99") * self.ema_rate <= self.spot_rate <= Decimal("1.01") * self.ema_rate

    @property
    def avg_tkn_trading_liquidity(self):
        """Computes the tkn trading liquidity, adjusted by the ema."""
        return self.bnt_trading_liquidity / self.ema_rate

    @property
    def tkn_excess(self):
        """Computes the difference between the masterVault tknBalance and the average averageTknTradingLiquidity."""
        return self.vault_tkn - self.avg_tkn_trading_liquidity

    @property
    def tkn_excess_bnt_equivalence(self):
        """Computes the equivalent bnt value of the non-trading tkn balance of the masterVault."""
        return self.tkn_excess * self.ema_rate

    @property
    def bnt_bootstrap_liquidity(self):
        """Computes the bntMinLiquidity multiplied by 2."""
        return 2 * self.bnt_min_liquidity

    @property
    def ema(self) -> Fraction:
        """Returns a fraction as two separate outputs"""
        return Fraction(self.ema_rate)

    @property
    def scaled_ema(self) -> int:
        """Ensures that the ema can be stored as a maximum 112-bit number."""
        return ceil((max(self.ema.numerator, self.ema.denominator) / (2 ** 112 - 1)))

    @property
    def ema_compressed_numerator(self) -> int:
        """Used to measure the deviation of solidity fixed point math on protocol calclulations."""
        if self.scaled_ema > 0:
            return int(self.ema.numerator / self.scaled_ema)
        else:
            return 0

    @property
    def ema_compressed_denominator(self) -> int:
        """Used to measure the deviation of solidity fixed point math on protocol calclulations."""
        if self.scaled_ema > 0:
            return int(self.ema.denominator / self.scaled_ema)
        else:
            return 0

    @property
    def is_ema_update_allowed(self) -> bool:
        """Returns True if the moving average has not been updated on the existing block."""
        return int(self.timestep) != int(self.ema_last_updated)

    @property
    def ema_deviation(self) -> Decimal:
        """Returns the deviation between these values as emaRate/emaCompressedRate."""
        if self.ema_compressed_numerator > 0:
            return self.ema_rate * Decimal(self.ema_compressed_denominator / self.ema_compressed_numerator)
        else:
            return Decimal('0')


In a later chapter we will discuss the other `Ledger` classes of the protocol.